<a href="https://colab.research.google.com/github/justin95214/logistics_colab_code/blob/main/(MFC)SCH_%EB%B0%B0%EC%B0%A8_%EC%B5%9C%EC%8B%A0%EC%97%85%EB%8D%B0%EC%9D%B4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import sys
import numpy as np
import pandas as pd
from dataclasses import dataclass, asdict
import io
from google.colab import files
import folium
import random
import re

### 데이터 업로드

In [2]:
# client.xlsx 파일 읽기

client_data = files.upload()
client_data = pd.DataFrame()
try:
    client_data = pd.read_excel(io.BytesIO(client_data[url]))
except:
    client_data = pd.read_excel('/content/client_data.xlsx')

Saving client_data.xlsx to client_data.xlsx


In [3]:
# center.xlsx 파일 읽기

center_data = files.upload()
center_data = pd.DataFrame()
try:
    center_data = pd.read_excel(io.BytesIO(center_data[url]))
except:
    center_data = pd.read_excel('/content/center.xlsx')

Saving center.xlsx to center.xlsx


In [4]:
# Df_info.xlsx 파일 읽기

Df_info_data = files.upload()
Df_info_data = pd.DataFrame()
try:
    Df_info_data = pd.read_excel(io.BytesIO(Df_info_data[url]))
except:
    Df_info_data = pd.read_excel('/content/condition.xlsx')

Saving condition.xlsx to condition.xlsx


### 배차 스케줄링 기본 함수가 내재된 관련 클래스 선언

##### 배차 스케줄링에 있어, 엑셀파일에서 데이터프레임 형식으로 데이터를 가져오고, 곡률값 계산, 방위각 계산, DP와 클라이언트의 거리, 클라이언트에서 클라이언트의 거리 구 하는 함수가 담긴 클래스를 선언

In [5]:
class sch_sys():
	
	def __init__(self, client, dp, df_info):
		self.client = client
		self.dp = dp
		self.df_info = df_info


	#데이터 로드
	def load_data(self):
		df_client = pd.read_excel(self.client)
		df_DP = pd.read_excel(self.dp)
		df_info = pd.read_excel(self.df_info)
		#tt.autolog()
		return df_client, df_DP, df_info

	#데이터프레임 병합	
	def merge_data(self, client, dp, how_way):
		df_merge = pd.merge(left=client, right=dp, on="DP", how= how_way)
		#tt.autolog()
		return df_merge

	# First time일 때 DP의 곡률값계산
	def cal_xy_DP(self, temp_df):
		temp_df['DP_y곡률값']=temp_df['Latitude']*3600
		temp_df['DP_x곡률값']=temp_df['Longitude']*3600
		#tt.autolog()
		return temp_df


	# First time일 때 client의 곡률값
	def cal_xy_client(self, temp_df):
		
		temp_df['x곡률값']=temp_df['경도(X좌표)']*3600
		temp_df['y곡률값']=temp_df['위도(Y좌표)']*3600
		#tt.autolog()
		return temp_df


	#client - client 상황일 때 - 곡률값계산
	def cal_xy_point(self, temp_df, current_point):
		y = temp_df[temp_df['code']==current_point]['경도(X좌표)'].values.tolist()[0]
		x = temp_df[temp_df['code']==current_point]['위도(Y좌표)'].values.tolist()[0]

		temp_df[current_point+'_y곡률값']=x*3600
		temp_df[current_point+'_x곡률값']=y*3600
		#print(temp_df.head(5))
		return temp_df


	#현재 위치에서 거래처간 거리 계산
	def cal_xy_distance(self, temp_df, current_point):
		#print(temp_df.head(3))
		temp_df[current_point+'_경도거리']= abs(temp_df['x곡률값']-temp_df[current_point+'_x곡률값'])*0.0245
		temp_df[current_point+'_위도거리']= abs(temp_df['y곡률값']-temp_df[current_point+'_y곡률값'])*0.0306
		temp_df[current_point+'_운행거리'] = temp_df[current_point+'_위도거리']+temp_df[current_point+'_경도거리']
		temp_df[current_point+'_위도거리'] = temp_df[current_point+'_위도거리'].astype(float)
		temp_df[current_point+'_경도거리'] = temp_df[current_point+'_경도거리'].astype(float)
		return temp_df

	
	#데이터프레임 초기화
	def dateframe_init(self, df_client):
		temp_df = df_client.loc[df_client['수량']!= np.NaN].copy()
		temp_df['stopby'] = 0
		return temp_df

	#데이터프레임 방위각 계산
	def cal_azimuth(self, temp_df, current_point):

		CPI = math.pi
		CstLatVal = 0.0245
		CstLonVal = 0.0306
		CStVal = 3600
		#temp_df =temp_df.dropna(axis=0)
		temp_df['rad'] = np.arctan2(temp_df[current_point+'_위도거리'],temp_df[current_point+'_경도거리']) 
		temp_df['deg'] = temp_df['rad'] * 180 / CPI
		#tt.autolog()
		return temp_df


### 데이터 객체

In [6]:
@dataclass
class Element:
	left_kappa: int =None
	kappa: int =None
	trace_list: list = None
	average_speed: float = None
	left_drive_time: float =None
	drive_time: float = None
	get_up: int =None
	get_off: int = None
	distance_list: list = None
	time_list: list = None
	total_distance: float = None
	current_point: str =None



### Route 라우팅 함수와 라우팅에 필요한 함수들이 들어있는 Route 클래스 선언

#### sch_sys클래스의 함수들을 기반으로 한 라우트에서 처음 DP에서 출발할 때와 Client - Client끼리 갈때, 그 상황을 업데이트 해주는 함수 들이 내재되어있다.

In [7]:
class Route:
	def __init__(self, client, dp, df_info,temp_df):
		self.temp_df = temp_df
		self.dp = dp
		self.df_info = df_info
		self.client = client
		self.tool = sch_sys(self.client, self.dp, self.df_info)
		self.element = Element()
	
	# route의 첫번째돌때
	def first_time(self, temp_df, current_point):
		temp_df = self.tool.cal_xy_DP(temp_df)
		temp_df = self.tool.cal_xy_client(temp_df)
		temp_df = self.tool.cal_xy_distance(temp_df, current_point)
		temp_df = temp_df[temp_df['stopby']==0].sort_values(by=[current_point+'_운행거리'], ascending=[False])
		return temp_df

  # client - client  case
	def client_to_client(self, temp_df, current_point):	 
		temp_df = self.tool.cal_xy_point(temp_df, str(current_point))
		temp_df = self.tool.cal_xy_distance(temp_df, current_point)
		temp_df = self.tool.cal_azimuth(temp_df, current_point)
		return temp_df

  # initize the dataframe
	# kappa, 주어진 시간, 평균 운행 속도, 상 하차 시간
	def update_init_element(self, df_info, df_element):
		self.element.kappa = df_info[df_info['DP']==df_element]['대당 적재능력(box)'].values.tolist()[0]
		self.element.drive_time = df_info[df_info['DP']==df_element]['대당 운행시간(분)'].values.tolist()[0]	
		self.element.average_speed = df_info[df_info['DP']==df_element]['평균 운행 속도(km/h)'].values.tolist()[0]
		self.element.get_up = df_info[df_info['DP']==df_element]['상차시간(분)'].values.tolist()[0]
		self.element.get_off = df_info[df_info['DP']==df_element]['하차시간(분)'].values.tolist()[0]
		#첫 리스트 포인트
		self.element.time_list = [str(self.element.drive_time)]
		self.element.trace_list =['DP']
		self.element.distance_list = ['0']
		self.element.total_distance = 0
		self.element.current_point = self.element.trace_list[-1]
		self.element.left_drive_time = self.element.drive_time
		self.element.left_kappa = self.element.kappa


	def situation_update(self, Max_point, temp_df, df_element, result_list):
		print(self.element)
		# 이번 client 이동시 발생하는 운행 거리
		this_distance = Max_point[self.element.trace_list[-1]+'_운행거리']
		# 이번 client 이동시 소요 시간 = 운행 거리 / 평균 속도(km/h)
		this_time = this_distance / self.element.average_speed
		# element(dictclass 변수 집합)의 운전시간 값 = min 분 단위로 변경 x60분
		self.element.drive_time = this_time*60
		# 누적으로 남는 운전 시간 = element(dictclass 변수 집합)의 left_drive_time
		before_tmp = self.element.left_drive_time

		# 업데이트 된 누적 남는  운전시간 = 이전에 남은 운전시간 - 이번 운전 시간 - 하차 시간 
		self.element.left_drive_time = self.element.left_drive_time - this_time*60 - self.element.get_off

		# 남은 운전시간이 0 보다 크면  
		if self.element.left_drive_time >0:
			print('# 남은 운전시간이 0 보다 크면  ')
			# >> 차량 케파 - 이번 클라이언트 수량
			self.element.left_kappa = self.element.left_kappa - Max_point['수량']
			
			# 근데 업데이트 된 차량 케파가 0보다 작으면 
			if self.element.left_kappa <= 0:
				print('# 근데 업데이트 된 차량 케파가 0보다 작으면 ')
				#이전상황으로 운행거리, 시간, kappa 복귀 = >> 차량 케파 + 이번 클라이언트 수량
				self.element.left_kappa = self.element.left_kappa + Max_point['수량']
				
				# 현재 상황의 element을 dict형태로 변환
				tmp_dict = asdict(self.element)
				# dict형태로 변환 현재 상황을 리스트화
				tmp_list = list(tmp_dict.values())

				tmp_list.append(len(tmp_list[2])-1)
				result_list.append(tmp_list)

				#다시 route를 돌리기 위한 현재 상황 초기화
				self.update_init_element(self.df_info, df_element)
				
			# 근데 업데이트 된 차량 케파가 0보다 크면
			else:	
				print('# 근데 업데이트 된 차량 케파가 0보다 크면')
				# 총 이동거리 업데이트
				self.element.total_distance = self.element.total_distance + this_distance
				self.element.distance_list.append(this_distance)
		
				# 남은 시간 업데이트
				self.element.time_list.append(str(float(self.element.time_list[-1]) - self.element.drive_time))
		
				# 이동 경로 업데이트
				self.element.trace_list.append(Max_point['code'])
		
				# 현재 위치 업데이트
				self.element.current_point = self.element.trace_list[-1]

		# 케파는 있지만 마지막 행선지를 다녀왔을 때 근무시간이 오버되더라도 마지막은 다녀온다는 가정
		else:
			print('# 케파는 있지만 마지막 행선지를 다녀왔을 때 근무시간이 오버되더라도 마지막은 다녀온다는 가정')
			# 수량, 거리, 현재 위치, 이동 경로 업데이트, 방문 stopby 업데이트
			self.element.left_kappa = self.element.left_kappa - Max_point['수량']
			self.element.total_distance = self.element.total_distance + this_distance
			self.element.distance_list.append(this_distance)
			self.element.time_list.append(str(float(self.element.time_list[-1]) - self.element.drive_time))
			self.element.trace_list.append(Max_point['code'])
			self.element.current_point = self.element.trace_list[-1]

			temp_df.loc[temp_df['code']==Max_point['code'],'stopby']=1

			# 현재 상황의 element을 dict형태로 변환
			tmp_dict = asdict(self.element)
			tmp_list = list(tmp_dict.values())
			tmp_list.append(len(tmp_list[2])-1)
			
			result_list.append(tmp_list)

			#다시 route를 돌리기 위한 현재 상황 초기화
			self.update_init_element(self.df_info, df_element)


	# route 전체
	def route(self, temp_df, df_element,  current_point, df_info):
		
		result_df = pd.DataFrame()		

		#총 루트
		route_list = []	
		
		#차량 조건 초기화
		self.update_init_element(df_info, df_element)	
		#클라이언트 개수
		data_count = len(temp_df[temp_df['stopby']==0])

		# stopby =0 인 것이 없을 때까지 while문 돌기
		while(len(temp_df[temp_df['stopby']==0]) !=0):
			#print(temp_df.head(5))
			# 차량 남은 시간이 적어 새 차량을 배차시킬 때 와 첫 출발 일때

			# 총거리가 0으로 초기화 될때 
			if self.element.total_distance == 0:
				temp_df = self.first_time(temp_df, self.element.current_point)

				# 처음은 거리로 정렬
				Max_distance_row_point = temp_df.iloc[0]
				
				#업데이트 - 여기서 마지막이 될 수 도 잇음
				self.situation_update(Max_distance_row_point, temp_df, df_element, route_list)			

			# stopby 가 1개 이상 남을 때까지 돌기 그리고 총거리가 0이 아닐 때 돌게 됨
			else:
				if len(temp_df[temp_df['stopby']==0]) >=1:

					temp_df = self.client_to_client(temp_df, self.element.current_point)
					
					# stopby = 1로 갔다는 것을체크
					temp_df.loc[temp_df['code']==Max_distance_row_point['code'],'stopby']=1
					# 각도와 거리의 가중치 합으로 정렬
					temp_df['new_std'] = w1 * temp_df['deg'] + w2 * temp_df[self.element.current_point+'_운행거리']
					temp_df = temp_df.sort_values(by=['new_std'], ascending=[True])				

					#temp_df = temp_df.sort_values(by=['deg'], ascending=[True])
					#print(len(temp_df[temp_df['stopby']==0]))

					# stopby =0 일 때 중간 종료
					if len(temp_df[temp_df['stopby']==0]) ==0:
						
						tmp_dict = asdict(self.element)
						tmp_list = list(tmp_dict.values())  
						tmp_list.append(len(tmp_list[2])-1)
						route_list.append(tmp_list)

						result_df = pd.DataFrame(route_list)
						result_df_columns = list(tmp_dict.keys())
						result_df_columns.append('count')					
						result_df.columns = result_df_columns

						result_df.to_csv("111result.csv", encoding='utf-8-sig')	
						break;

					Max_distance_row_point = temp_df[temp_df['stopby']==0].iloc[0]

					# 여기서 마지막이 될 수 도 있음         
					self.situation_update(Max_distance_row_point, temp_df, df_element, route_list)	

		return result_df


### 지도 그리는 함수

In [8]:


def random_color_code():
	list_A = [ i for i in range(1,10)]
	out = random.sample(list_A, 6)
	out = list(map(str, out))
	return out


def vlookup(df1, df2, df3):
 
	df1_list =[]
	color_listA= []	

	truck_number_data = df1['truck_number'].values.tolist()
	trace_list_data = df1['trace_list'].values.tolist()
	tmp_data = []


	for idx, row in enumerate(trace_list_data):
		tmp_data.extend(['truck'+str(idx+1)]*len(row))
		for i in row:
			df1_list.append(i)	


	df_data1 = pd.DataFrame(zip(tmp_data, df1_list))
	df_data1.columns = ['truck_number','code']

	df_data1 = df_data1.join(df2.set_index('code')['경도(X좌표)'], on='code')
	df_data1 = df_data1.join(df2.set_index('code')['위도(Y좌표)'], on='code')

	df_data1.loc[df_data1['code'] =='DP', '경도(X좌표)'] = df3['Longitude'].values.tolist()[0]
	df_data1.loc[df_data1['code'] =='DP', '위도(Y좌표)'] = df3['Latitude'].values.tolist()[0]
	

	df_data1.to_csv("tmp_result.csv", encoding='utf-8-sig')

	init_xy = [df3['Latitude'].values.tolist()[0] ,df3['Longitude'].values.tolist()[0]]

	MM = folium.Map(location = init_xy, zoom_start=10)
	
	position_name = df_data1['code'].values.tolist()
	position_truck = df_data1['truck_number'].values.tolist()
	position_list = df_data1[['위도(Y좌표)','경도(X좌표)']].values.tolist()

	color_list = random_color_code()
	color_code = ''.join(color_list)
	color_code = '#' + color_code
	
	color_list = ['blue','green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 
'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

	for P, T, M in zip(position_list, position_truck ,position_name):
		numbers = re.sub(r'[^0-9]', '', T)
		if M != 'DP':

			folium.Marker(P, popup= T + "-" + M, icon=folium.Icon(color=color_list[int(numbers) % len(color_list)])).add_to(MM)
		else:
			folium.Marker(P, popup= T + "-" + M, icon=folium.Icon(color='red')).add_to(MM)

	folium.PolyLine(locations=position_list, tooltip='Polyline').add_to(MM)
	folium.Polygon(locations=position_list, fill=True, tooltip='Polygon').add_to(MM)

	MM.save("map_test.html")
        


In [9]:
url = './'
DF_client = url+'client_data.xlsx'
DP = url+'center.xlsx'
Df_info = url+'condition.xlsx'

# 정렬 가중치 적용 w1 / w2
w1 = 1
w2 = 5

### 메인함수

In [11]:
client = DF_client
dp = DP
df_info = Df_info

# 배차 스케줄링 기본 함수가 내재된 관련 클래스 객체 선언
sch = sch_sys(client, dp, df_info)

# 3가지 엑셀 데이터 로드
df_client, df_DP, df_info = sch.load_data()

# 클라리언트 데이터와 DP센터 데이터 병합
df_merge = sch.merge_data(df_client, df_DP, "right")
df_client = df_merge.copy()



# 시계 계산을 위한 apply함수용 함수
def applyfunc(x):
	return float(x[0]) - float(x[-1])
		

#for DF_element in df_DP['DP'].values.tolist():

# 적용하고자 하는 지역은 안산 센터 데이터 가져오기
DF_element = df_DP['DP'].values.tolist()[0]

#시작은 DP임으로 현재 지점은 DP 이다
current_point = 'DP'

# 병합 데이터 초기화 - 수량이 없는 데이터는  제외 / stopby = 0으로 들렸는 지 확인하는 컬럼 추가
temp_df = sch.dateframe_init(df_client)
temp_df = temp_df.dropna(subset=['수량'], how='any', axis = 0)	

# 라우트 객체 선언
route = Route(client, dp, df_info, temp_df)

# 라우트 함수 실행
raw_result = route.route(temp_df, DF_element, current_point, df_info)

# 결과 엑셀 데이터 출력을 위한 정리 과정
raw_result['total_time'] = raw_result['time_list'].apply(applyfunc)
raw_result.drop(['kappa','average_speed','left_drive_time','drive_time','get_up','get_off','distance_list','time_list','current_point'], axis=1)
raw_result['truck_number'] = pd.DataFrame(['truck'+str(i+1) for i in range(len(raw_result))])

# 결과 엑셀데이터에 사용할 컬럼
raw_result = raw_result[['truck_number','total_distance','total_time','left_kappa','count','trace_list']]

# 결과 엑셀 데이터 출력
raw_result.to_csv("result.csv", encoding='utf-8-sig', index=False)	
# 지도 그려주기
vlookup(raw_result, df_client, df_DP)


Element(left_kappa=80, kappa=80, trace_list=['DP'], average_speed=35, left_drive_time=480, drive_time=480, get_up=20, get_off=5, distance_list=['0'], time_list=['480'], total_distance=0, current_point='DP')
# 남은 운전시간이 0 보다 크면  
# 근데 업데이트 된 차량 케파가 0보다 크면
Element(left_kappa=77.0, kappa=80, trace_list=['DP', '안산-2'], average_speed=35, left_drive_time=280.7865264035771, drive_time=194.2134735964229, get_up=20, get_off=5, distance_list=['0', 113.29119293124668], time_list=['480', '285.7865264035771'], total_distance=113.29119293124668, current_point='안산-2')
# 남은 운전시간이 0 보다 크면  
# 근데 업데이트 된 차량 케파가 0보다 크면
Element(left_kappa=74.0, kappa=80, trace_list=['DP', '안산-2', '안산-210'], average_speed=35, left_drive_time=227.382421838359, drive_time=48.40410456521812, get_up=20, get_off=5, distance_list=['0', 113.29119293124668, 28.235727663043903], time_list=['480', '285.7865264035771', '237.382421838359'], total_distance=141.52692059429057, current_point='안산-210')
# 남은 운전시간이 0 보다 크면  
# 근데 업데이트 된 차량 케파